In [78]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [86]:
import tensorflow as tf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import cm
import os
import distance
%matplotlib inline
sns.set(rc={'axes.facecolor':'#FFFFFF'})
sns.set_style('whitegrid')
pd.set_option('display.max_columns', 100)

## 1. Data formatting

Reading in data. Filtering data. Transforming it into samples/features format. Mutation positions according to Scer. Writing to file.

### 1.1. Segment-based numeration

In [92]:
data = {}
clean_data = {}
mut_data = {}

for f in os.listdir('/home/katya/start/HIS3InterspeciesEpistasis/Data/'):
    if 'csv' in f:
        data[f[:-19]] = pd.DataFrame.from_csv('/home/katya/start/HIS3InterspeciesEpistasis/Data/' + f, sep='\t')
        median = np.median(data[f[:-19]].index.str.len())
        
        clean_data[f[:-19]] = data[f[:-19]][(data[f[:-19]].nonsense == 0) & (data[f[:-19]].middle == 1) & \
                                            (data[f[:-19]].nat_lib == 1) & (data[f[:-19]].stop == 0) & \
                                            (data[f[:-19]].index.str.len() == median)]
        
        mut_data[f[:-19]] = pd.DataFrame(clean_data[f[:-19]].mut_list)
        mut_data[f[:-19]]['fitness'] = clean_data[f[:-19]].s
        mut_data[f[:-19]].index = mut_data[f[:-19]].mut_list
        mut_data[f[:-19]].drop(['mut_list'], axis=1, inplace=True)
        mut_data[f[:-19]]['aa_seq'] = clean_data[f[:-19]].index
        
        mut_data[f[:-19]].to_csv('/home/katya/start/HIS3InterspeciesEpistasis/Analysis/Katya/NN/data/' + f[:-19] + '.txt', sep='\t')

### 1.2. Scer-based numeration

In [34]:
data = {}
clean_data = {}
mut_data = {}

for f in os.listdir('/home/katya/start/HIS3InterspeciesEpistasis/Data/'):
    if 'csv' in f:
        data[f[:-19]] = pd.DataFrame.from_csv('/home/katya/start/HIS3InterspeciesEpistasis/Data/' + f, sep='\t')
        median = np.median(data[f[:-19]].index.str.len())
        
        clean_data[f[:-19]] = data[f[:-19]][(data[f[:-19]].nonsense == 0) & (data[f[:-19]].middle == 1) & \
                                            (data[f[:-19]].nat_lib == 1) & (data[f[:-19]].stop == 0) & \
                                            (data[f[:-19]].index.str.len() == median)]
        
        mut_data[f[:-19]] = pd.DataFrame(clean_data[f[:-19]].mut_list_Scer)
        mut_data[f[:-19]]['fitness'] = clean_data[f[:-19]].s
        mut_data[f[:-19]].index = mut_data[f[:-19]].mut_list_Scer
        mut_data[f[:-19]].drop(['mut_list_Scer'], axis=1, inplace=True)
        mut_data[f[:-19]]['aa_seq'] = clean_data[f[:-19]].index
        
        mut_data[f[:-19]].to_csv('/home/katya/start/HIS3InterspeciesEpistasis/Analysis/Katya/NN/data/' + f[:-19] + '.txt', sep='\t')

In [76]:
mask = clean_data['S9'].mut_list.str.contains('5')
mask = mask.fillna(False)
filtered = clean_data['S9'][mask]
len(filtered[filtered.s==0])

2530

In [70]:
len(clean_data['S9'][clean_data['S9'].s==0])

4644

## 2. Testing

In [9]:
for chunk in clean_data:
    print chunk
    print mut_data[chunk].shape

S9
(30862, 1)
S8
(46328, 1)
S3
(48530, 1)
S12
(33275, 1)
S1
(58147, 1)
S10
(29965, 1)
S7
(4315, 1)
S6
(45635, 1)
S5
(51319, 1)
S4
(44368, 1)
S2
(46229, 1)
S11
(16857, 1)


In [67]:
FreqDist((':').join(clean_data['S4'].mut_list_Scer).split(':'))

FreqDist({'': 62,
          '171A': 34486,
          '171C': 2,
          '171E': 23,
          '171G': 3,
          '171L': 8,
          '171T': 24,
          '171V': 11,
          '172A': 14217,
          '172C': 16,
          '172D': 48,
          '172E': 30,
          '172G': 13510,
          '172I': 41,
          '172K': 61,
          '172M': 28,
          '172N': 62,
          '172S': 12655,
          '172T': 14267,
          '172V': 42,
          '172W': 4,
          '173A': 2,
          '173C': 1,
          '173E': 4,
          '173G': 1,
          '173H': 1,
          '173K': 6,
          '173L': 126,
          '173M': 28,
          '173T': 2,
          '173V': 28245,
          '174A': 9,
          '174I': 11,
          '174K': 25,
          '174M': 1,
          '174N': 48,
          '174S': 4,
          '175A': 1,
          '175F': 9,
          '175I': 19006,
          '175K': 2,
          '175M': 17220,
          '175P': 2,
          '175Q': 6,
          '175S': 1,
         

In [10]:
mut_data['S1']

,fitness
mut_list_Scer,
106D:108I:111F:112K:114I:115A:135F,0.431578
106D:108I:111F:112K:114I:115A,0.442993
106D:108I:111F:112K:114I:115A:131S:135F,0.199384
106D:108I:111F:112K:114I:115A:131S,0.315030
106D:108I:111F:112K:114I:115E:135F,0.416220
106D:108I:111F:112K:114I:115E,0.400086
106D:108I:111F:112K:114I:115E:131S:135F,0.056902
106D:108I:111F:112K:114I:115E:131S,0.159167
106D:108I:111F:112K:114I:135F,0.433622
